In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# Change to the directory containing CSV files
csv_folder = '/content/drive/MyDrive/F1_data_project'
os.chdir(csv_folder)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

FastLap_df = pd.read_csv('Fastest_Laps.csv')
FastLap_df.head()


In [ ]:
FastLap_df.drop(columns=['PitOutTime', 'PitInTime'], inplace=True)
FastLap_df.head()

In [ ]:
fastest_lap = FastLap_df.loc[FastLap_df['LapTime'].idxmin()]
fastest_lap

In [ ]:
fastest_lap['DriverNumber']

In [ ]:
FastLap_df.columns

Objectives with EDA:
1. Merge/join laps and fastest laps so we can identify drivers distribution of which lap number they tend to have fastest. Identifying patterns in fastest laptime

2. Preprocess all the time variables and make them more easier to work with

3. Decide how we are going to deal with telemetry data which seems to be very sparse

5. Observe Weather conditions for each of the laps and then look at conditions for fastest laps



In [ ]:
lap_df = pd.read_csv('Laps.csv')
lap_df.head()

In [ ]:
lap_df.columns

In [ ]:
lap_data = pd.concat([FastLap_df, lap_df], axis=0, ignore_index=True)
lap_data.head()

In [ ]:
#lap_data['Time'] = pd.to_timedelta(lap_data['Time'])
lap_data['LapTime'] = pd.to_timedelta(lap_data['LapTime'])
lap_data['normalized_t'] = lap_data['LapTime'].dt.total_seconds()
lap_data.head()

In [ ]:
lap_data['DriverNumber'].value_counts()

In [ ]:
lap_data['Compound'].value_counts()

In [ ]:
lap_data['TrackStatus'].value_counts()

In [ ]:
def fast_lapnum(subdf):
  # Convert 'LapTime' to numeric (total seconds) before finding the minimum
  subdf['LapTime'] = pd.to_timedelta(subdf['LapTime'])  # Ensure 'LapTime' is Timedelta
  subdf['LapTimeSeconds'] = subdf['LapTime'].dt.total_seconds() # Create a new column with LapTime in seconds
  return subdf.loc[subdf['LapTimeSeconds'].idxmin()] # Find the minimum based on the numeric column


In [ ]:
# subdataframes for each driver
driver_subgroup = lap_data.groupby('DriverNumber')
for driver, driver_subdf in driver_subgroup:
  # Create a copy of the sub-DataFrame to avoid modifying the original
  temp_subdf = driver_subdf.copy()

  # Recalculate normalized_t for the temporary sub-DataFrame
  temp_subdf['LapTime'] = pd.to_timedelta(temp_subdf['LapTime'])  # Ensure 'LapTime' is Timedelta
  temp_subdf['normalized_t'] = temp_subdf['LapTime'].dt.total_seconds()

  fast_laptime = fast_lapnum(temp_subdf) # Pass the copy to fast_lapnum

  # Use temp_subdf for plotting since it now contains 'normalized_t'
  plt.plot(temp_subdf['LapNumber'], temp_subdf['normalized_t'], label=f'Driver {driver}')
  plt.scatter(fast_laptime['LapNumber'], fast_laptime['normalized_t'], color='red', label=f'Fastest Laptime for Driver {driver}')
  plt.xlabel('Lap Number')
  plt.ylabel('Normalized Lap Time')
  plt.title(f'Lap Times for Driver {driver}')
  plt.show()


In [ ]:
# subdataframes for each driver
event_driver_group = lap_data.groupby(['EventName', 'DriverNumber'])
for (event, driver) , group_data in event_driver_group:
  # Create a copy of the sub-DataFrame to avoid modifying the original
  temp_subdf = driver_subdf.copy()

  # Recalculate normalized_t for the temporary sub-DataFrame
  temp_subdf['LapTime'] = pd.to_timedelta(temp_subdf['LapTime'])  # Ensure 'LapTime' is Timedelta
  temp_subdf['normalized_t'] = temp_subdf['LapTime'].dt.total_seconds()

  fast_laptime = fast_lapnum(temp_subdf) # Pass the copy to fast_lapnum

  # Use temp_subdf for plotting since it now contains 'normalized_t'
  plt.plot(temp_subdf['LapNumber'], temp_subdf['normalized_t'], label=f'Driver {driver}')
  plt.scatter(fast_laptime['LapNumber'], fast_laptime['normalized_t'], color='red', label=f'Fastest Laptime for Driver {driver}')
  plt.xlabel('Lap Number')
  plt.ylabel('Normalized Lap Time')
  plt.title(f'Lap Times for Driver {driver} at {event}')
  plt.show()


In [ ]:
def fast_lapnum(subdf):
  # Convert 'LapTime' to numeric (total seconds) before finding the minimum
  subdf['LapTime'] = pd.to_timedelta(subdf['LapTime'])  # Ensure 'LapTime' is Timedelta
  subdf['LapTimeSeconds'] = subdf['LapTime'].dt.total_seconds() # Create a new column with LapTime in seconds

  # Handle potential NaN values in 'LapTimeSeconds'
  if pd.isnull(subdf['LapTimeSeconds'].idxmin()):
    return None  # Or raise an exception, or return a default value
  else:
    return subdf.loc[subdf['LapTimeSeconds'].idxmin()] # Find the minimum based on the numeric column

# subdataframes for each driver in each event
event_driver_group = lap_data.groupby(['EventName', 'DriverNumber'])
for (event, driver), group_data in event_driver_group:

  temp_subdf = group_data.copy()

  # Recalculate normalized_t for the temporary sub-DataFrame
  temp_subdf['LapTime'] = pd.to_timedelta(temp_subdf['LapTime'])  # Ensure 'LapTime' is Timedelta
  temp_subdf['normalized_t'] = temp_subdf['LapTime'].dt.total_seconds()

  fast_laptime = fast_lapnum(temp_subdf) # Pass the copy to fast_lapnum

  # Check if fast_laptime is None before plotting
  if fast_laptime is not None:
    plt.plot(temp_subdf['LapNumber'], temp_subdf['normalized_t'], label=f'Driver {driver}')
    plt.scatter(fast_laptime['LapNumber'], fast_laptime['normalized_t'], color='red', label=f'Fastest Laptime for Driver {driver}')
    plt.xlabel('Lap Number')
    plt.ylabel('Normalized Lap Time')
    plt.title(f'Lap Times for Driver {driver} at {event}')
    plt.show()
  else:
    print(f"No valid lap times found for Driver {driver} at {event}") # Print a message or handle the case as needed

In [ ]:
# Convert the 'LapTime' column to timedelta, coerce errors to NaT
lap_data['LapTime'] = pd.to_timedelta(lap_data['LapTime'], errors='coerce')

# Drop rows with NaT (Not a Time) values in the 'LapTime' column
lap_data = lap_data.dropna(subset=['LapTime'])

# Now can find the fastest and slowest lap times
fastest_fastlap = lap_data.loc[lap_data['LapTime'].idxmin()]
slowest_fastlap = lap_data.loc[lap_data['LapTime'].idxmax()]
fastest_fastlap, slowest_fastlap

1. fastest_fastlap (the absolute fastest lap):

Benchmark: Represents the ultimate performance limit observed within the dataset. It can serve as a benchmark to compare other laps or drivers against.

Track Conditions and Driver Skill: Provides insights into the ideal combination of track conditions and driver skill that led to this exceptional lap time. Analyzing the associated data points (driver, lap number, weather conditions, etc.) can help identify key factors contributing to peak performance.

Outlier Detection: If the fastest_fastlap significantly deviates from other fast laps, it could indicate an outlier or an unusual event during the race (e.g., a sudden improvement in track conditions). This requires further investigation.

2. slowest_fastlap (the slowest among fastest laps):

Performance Variation: Highlights the variability in fastest lap times across drivers or race conditions. It indicates the lower end of the performance spectrum for "fastest" laps.

Identifying Limiting Factors: By analyzing the data points associated with the slowest_fastlap (driver, lap number, weather, tire compound, etc.), you can potentially identify factors that limited performance in that particular instance (e.g., tire degradation, traffic, unfavorable weather).

Strategy Insights: In some cases, a slower "fastest" lap might be a strategic choice, such as conserving tires or managing fuel. Comparing this lap to other laps by the same driver can provide context and insight into their race strategy.

Further Analysis
Here are some ways you can use fastest_fastlap and slowest_fastlap in your analysis:

Normalization: You can normalize other lap times relative to the fastest_fastlap to get a sense of their performance as a percentage of the best possible time.
Performance Distribution: Compare the distribution of all fastest lap times to these two extremes to understand the range and central tendency of performance.
Correlation Analysis: Investigate how factors like weather, tire compound, and driver experience correlate with the lap times, focusing on how these factors differ between fastest_fastlap and slowest_fastlap.
Segmentation: Group drivers or laps based on their proximity to fastest_fastlap or slowest_fastlap to identify performance clusters or patterns.
Predictive Modeling: Use the information gleaned from these extreme laps as features in predictive models to forecast lap times or race outcomes.
By understanding and comparing these extreme cases, you can gain a more comprehensive understanding of the factors that influence performance in Formula 1 racing and use this knowledge for more detailed analysis and predictions. I hope this explanation is helpful!

Rate this answer
Yes this is the essence of the goal for this particular project. I definitely want to try some of these analyses further and I believe I am going in a right path.

That's excellent to hear! It sounds like you've got a solid grasp of the objectives and have a good plan for using fastest_fastlap and slowest_fastlap in your analysis.

I'm confident that exploring the ideas we discussed, such as normalization, performance distribution analysis, correlation analysis, and segmentation, will lead to valuable insights. Remember to keep in mind the context of these extreme laps and their potential strategic implications within a race.

For example, when analyzing performance distribution, you might consider creating visualizations (histograms, box plots) to see how the other fastest lap times cluster around these two extremes. This can help you understand the typical range of "fastest" lap times and identify potential outliers.

In your correlation analysis, don't hesitate to explore interactions between different factors. For instance, how does the impact of tire compound on lap times change depending on the weather conditions?

When segmenting drivers or laps, consider using clustering algorithms or simply grouping them based on quantiles of lap time relative to fastest_fastlap and slowest_fastlap. This can reveal interesting patterns and performance tiers.

And as you move towards predictive modeling, you can use the information from these extreme laps, along with other relevant features, to train models that predict lap times or race outcomes.

Remember to document your findings and interpretations clearly. This will help you build a strong narrative around your analysis and communicate your insights effectively.

I'm excited to see what you discover in your project! Feel free to reach out if you have any further questions or need guidance along the way. I'm here to support your exploration.

In [ ]:
# Outlier analysis
# boxplot of all fastest lap times
sns.boxplot(x=lap_data['LapTime'])
plt.title('Boxplot of All Fastest Lap Times')
plt.show()


In [ ]:
# calculating z-score of fastest_fastlap
from scipy import stats

# Get all lap times as a NumPy array, handling potential missing values
lap_times = lap_data['LapTime'].dt.total_seconds().values

# Remove NaN values before calculating z-scores
valid_lap_times = lap_times[~np.isnan(lap_times)]

# Calculate z-scores for valid lap times
z_scores = np.abs(stats.zscore(valid_lap_times))

# Get the index of the fastest lap within the original lap_times array
fastest_lap_index = lap_data['LapTime'].idxmin()

# Check if the fastest lap time is valid (not NaN)
if not np.isnan(lap_times[fastest_lap_index]):
  # Adjust the index to match the valid_lap_times array
  adjusted_index = np.where(lap_times == lap_times[fastest_lap_index])[0][0]
  # Get the z-score for the fastest lap
  fastest_lap_zscore = z_scores[adjusted_index - np.sum(np.isnan(lap_times[:adjusted_index]))]
else:
  # Handle the case where the fastest lap time is NaN
  fastest_lap_zscore = np.nan # or raise an exception

fastest_lap_zscore

This value supports the observation that there was no outlier visible in the boxplot to the left of the boxplot representing lower laptime

In [ ]:
# Adressing outliers to the left of the box plot (high laptime)
Q1 = lap_data['LapTime'].quantile(0.25)
Q3 = lap_data['LapTime'].quantile(0.75)
Q2 = lap_data['LapTime'].quantile(0.5)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
slow_outliers = lap_data[(lap_data['LapTime'] < lower_bound) | (lap_data['LapTime'] > upper_bound)]
non_slow_outliers = lap_data[(lap_data['LapTime'] >= Q2) & (lap_data['LapTime'] <= upper_bound)]
print(f"Number of outliers: {len(slow_outliers)}")

In [ ]:
PCA_df = lap_data.copy()




PCA_df.drop(columns=['Time', 'DriverNumber', 'LapStartTime', 'Team', 'Driver', 'IsAccurate', 'RoundNumber', 'EventName', 'PitOutTime', 'PitInTime', 'LapStartDate'], inplace=True)


# preprocessing steps
PCA_df['FreshTyre'] = PCA_df['FreshTyre'].astype(int)
PCA_df['IsPersonalBest'] = PCA_df['IsPersonalBest'].astype(int)
# non binary
compound_mapping = {'SOFT' : 0, 'MEDIUM': 1, 'HARD' : 2, 'INTERMEDIATE' : 3, 'WET' : 4}
PCA_df['Compound'] = PCA_df['Compound'].map(compound_mapping)

# trackstatus
track_status_map = {
        1: 'Green',
        2: 'Yellow',
        4: 'Red',
        5: 'SC',        # Safety Car
        6: 'VSC',       # Virtual Safety Car
        7: 'Double Yellow'
    }
PCA_df['TrackStatusCat'] = PCA_df['TrackStatus'].map(track_status_map).fillna('Mixed')
PCA_df = pd.get_dummies(PCA_df, columns=['TrackStatusCat'], prefix='Track')
#print("PCA_df.shape after one-hot encoding:", PCA_df.shape)

# ... (more preprocessing code) ...
# Convert 'object' type columns to numeric
object_columns = PCA_df.select_dtypes(include=['object']).columns
for col in object_columns:
    try:
        # Attempt to convert to timedelta, then to total seconds
        PCA_df[col] = pd.to_timedelta(PCA_df[col]).dt.total_seconds()
    except (ValueError, TypeError):
        # If timedelta conversion fails, try converting to numeric directly
        try:
            PCA_df[col] = pd.to_numeric(PCA_df[col], errors='coerce')
            # You might need to handle non-numeric values here (e.g., imputation or dropping rows)
        except ValueError:
            print(f"Column '{col}' could not be converted to numeric. Consider other methods.")

# Also, convert 'LapTime' to total seconds
PCA_df['LapTime'] = pd.to_timedelta(PCA_df['LapTime']).dt.total_seconds()
print("Missing values after all encoding and conversions:\n", PCA_df.isnull().sum())


PCA does not work if there are any missing values so needed to impute the means for speed features and compound which seemed to have all values missing due to the duplication

In [ ]:
# 1. Fix the Compound mapping
# Get unique values in the 'Compound' column of the original lap_data
unique_compounds = lap_data['Compound'].unique()

# Update the compound_mapping to include all unique values
compound_mapping = {compound: i for i, compound in enumerate(unique_compounds)}

# Re-apply the mapping
PCA_df['Compound'] = lap_data['Compound'].map(compound_mapping)


# 2. Handle missing values in other columns:
# Option 1: Imputation
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Or 'median', or another strategy
PCA_df[['SpeedI1', 'SpeedFL', 'SpeedST', 'Sector1Time', 'Sector1SessionTime']] = imputer.fit_transform(PCA_df[['SpeedI1', 'SpeedFL', 'SpeedST', 'Sector1Time', 'Sector1SessionTime']])

# Option 2: Removal (if data loss is acceptable)
# PCA_df.dropna(subset=['SpeedI1', 'SpeedFL', 'SpeedST', 'Sector1Time', 'Sector1SessionTime'], inplace=True)


# # 3. Remove duplicate columns
# # Get a list of all column names
# columns = PCA_df.columns.tolist()

# # Find and remove duplicate column names (keeping the first occurrence)
# seen_columns = set()
# columns_to_keep = []
# for column in columns:
#     if column not in seen_columns:
#         seen_columns.add(column)
#         columns_to_keep.append(column)

# # Select only the unique columns
# PCA_df = PCA_df[columns_to_keep]
print(PCA_df.isnull().sum())

In [ ]:
print(PCA_df.info())

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming PCA_df is your preprocessed DataFrame with all numerical columns

# 1. Create a PCA object
pca = PCA()  # You can specify n_components if you want to reduce to a specific number of components

# 2. Fit the PCA model to your data
pca.fit(PCA_df)

# 3. Get the principal components
principal_components = pca.transform(PCA_df)

In [ ]:
# Checking the ideal n_components using the explained variance ratio
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

## Want n_components to be 5 according to the Explained variance ratio plot

In [ ]:
pca5 = PCA(n_components=5)
principal_components5 = pca5.fit_transform(PCA_df)
loadings_df = pd.DataFrame(pca5.components_.T, columns=[f'PC{i+1}' for i in range(5)], index=PCA_df.columns)


In [ ]:
# Get the component loadings for PC1
pc1_loadings = loadings_df['PC1'].abs().sort_values(ascending=False)

# Select the top 5 features
top_5_features = pc1_loadings.head(5).index.tolist()

# Create the new DataFrame with selected features
new_df = PCA_df[top_5_features]

# Display the first few rows of the new DataFrame
print(new_df.head())

In [ ]:
slow_outliers_pca = slow_outliers[new_df.columns]
non_slow_outliers_pca = non_slow_outliers[new_df.columns]

In [ ]:
slow_outliers_pca['Sector1SessionTime'] = pd.to_timedelta(slow_outliers_pca['Sector1SessionTime']).dt.total_seconds()
slow_outliers_pca['Sector2SessionTime'] = pd.to_timedelta(slow_outliers_pca['Sector2SessionTime']).dt.total_seconds()
slow_outliers_pca['Sector3SessionTime'] = pd.to_timedelta(slow_outliers_pca['Sector3SessionTime']).dt.total_seconds()
non_slow_outliers_pca['Sector1SessionTime'] = pd.to_timedelta(non_slow_outliers_pca['Sector1SessionTime']).dt.total_seconds()
non_slow_outliers_pca['Sector2SessionTime'] = pd.to_timedelta(non_slow_outliers_pca['Sector2SessionTime']).dt.total_seconds()
non_slow_outliers_pca['Sector3SessionTime'] = pd.to_timedelta(non_slow_outliers_pca['Sector3SessionTime']).dt.total_seconds()
new_df['Sector1SessionTime'] = pd.to_timedelta(new_df['Sector1SessionTime']).dt.total_seconds()
new_df['Sector2SessionTime'] = pd.to_timedelta(new_df['Sector2SessionTime']).dt.total_seconds()
new_df['Sector3SessionTime'] = pd.to_timedelta(new_df['Sector3SessionTime']).dt.total_seconds()

In [ ]:
# Creating a dynamic clustering function to compare outlier and non outlier with respect to the 5 features above
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import itertools
from sklearn.mixture import GaussianMixture
# generating all the feature combinations from which clusters will be created
def feature_combinations(new_df, max_combination_size):
  """Generates combinations of features from a DataFrame.

    Args:
        df: The pandas DataFrame containing the features.
        max_combination_size: The maximum size of feature combinations
                               (default: 2, for pairs of features).

    Returns:
        A list of feature combinations (tuples).
    """
  features = new_df.columns.tolist()
  combinations = []
  for r in range(1, min(max_combination_size + 1, len(features) + 1)):
      combinations.extend(itertools.combinations(features, r))
  return combinations

def dynamic_clustering(cluster_type):
  if cluster_type==Km_clustering:
    for combination in feature_combinations(new_df, max_combination_size=2):
      # preprocess data
      scaler = StandardScaler()
      all_data = pd.concat([slow_outliers[list(combination)], non_slow_outliers[list(combination)]])
      all_data_scaled = scaler.fit_transform(all_data) # Fit on combined data

      Km_clustering = KMeans(n_clusters=2)  # Specify n_clusters=2
      all_data_clustered = Km_clustering.fit_predict(all_data_scaled)
      slow_outliers['cluster'] = all_data_clustered[:len(slow_outliers)]
      non_slow_outliers['cluster'] = all_data_clustered[len(slow_outliers):]
      # visualizing the cluster
       # Visualize clusters
       if len(combination) == 1:
          # Check if there's only one feature
        feature = combination[0]  # Get the feature name
        plt.figure(figsize=(8, 6))
        plt.hist(slow_outliers[feature], bins=20, alpha=0.5, label='Slow Outliers', color='red')
        plt.hist(non_slow_outliers[feature], bins=20, alpha=0.5, label='Non-Slow Outliers', color='blue')
        plt.title(f'Kmeans Distribution of {feature} for Outliers and Non-Outliers')
        plt.xlabel(feature)
        plt.ylabel('Frequency')
        plt.legend()
        cluster_centers = km_clustering.cluster_centers_
        plt.axvline(cluster_centers[0, 0], color='black', linestyle='--', label = 'cluster center')
        plt.axvline(cluster_centers[1, 0], color='black', linestyle='--', label = 'cluster center')
        plt.show()

      else:

        plt.figure(figsize=(8, 6))  # Adjust figure size as needed

      # Scatter plot for slow outliers
        plt.scatter(slow_outliers[slow_outliers['cluster'] == 0][list(combination)[0]],
                    slow_outliers[slow_outliers['cluster'] == 0][list(combination)[1]],
                    label='Slow Outliers Cluster 0', marker='o', color='red', alpha=0.7)
        plt.scatter(slow_outliers[slow_outliers['cluster'] == 1][list(combination)[0]],
                    slow_outliers[slow_outliers['cluster'] == 1][list(combination)[1]],
                    label='Slow Outliers Cluster 1', marker='x', color='red', alpha=0.7)

      # Scatter plot for non-slow outliers
        plt.scatter(non_slow_outliers[non_slow_outliers['cluster'] == 0][list(combination)[0]],
                    non_slow_outliers[non_slow_outliers['cluster'] == 0][list(combination)[1]],
                    label='Non-Slow Outliers Cluster 0', marker='o', color='blue', alpha=0.7)
        plt.scatter(non_slow_outliers[non_slow_outliers['cluster'] == 1][list(combination)[0]],
                    non_slow_outliers[non_slow_outliers['cluster'] == 1][list(combination)[1]],
                    label='Non-Slow Outliers Cluster 1', marker='x', color='blue', alpha=0.7)
        cluster_centers = km_clustering.cluster_centers_
        plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], marker='*', s=200, c='black', label='Cluster centers')
        plt.title(f'Kmeans Clustering for {list(combination)[0]} and {list(combination)[1]}')
        plt.xlabel(list(combination)[0])
        plt.ylabel(list(combination)[1])
        plt.legend()
        plt.show()
  elif cluster_type == gmm:
    for combination in feature_combinations(new_df, max_combination_size=2):

      all_data = pd.concat([slow_outliers[list(combination)], non_slow_outliers[list(combination)]])
      all_data_scaled = scaler.fit_transform(all_data) # Fit on combined data
      gmm = GaussianMixture(n_components=2, random_state=42)
      gmm.fit(all_data_scaled)
      cluster_labels = gmm.predict(all_data_scaled)
      slow_outliers['cluster'] = cluster_labels[:len(slow_outliers)]
      non_slow_outliers['cluster'] = cluster_labels[len(slow_outliers):]
        # visualizing the cluster
        if len(combination) == 1:
            # Check if there's only one feature
          feature = combination[0]  # Get the feature name
          plt.figure(figsize=(8, 6))
          plt.hist(slow_outliers[feature], bins=20, alpha=0.5, label='Slow Outliers', color='red')
          plt.hist(non_slow_outliers[feature], bins=20, alpha=0.5, label='Non-Slow Outliers', color='blue')
          plt.title(f'Distribution of {feature} for Outliers and Non-Outliers')
          plt.xlabel(feature)
          plt.ylabel('Frequency')
          plt.legend()
          cluster_centers = gmm.means_
          plt.axvline(cluster_centers[0, 0], color='black', linestyle='--', label = 'cluster center')
          plt.axvline(cluster_centers[1, 0], color='black', linestyle='--', label = 'cluster center')
          plt.show()
        else:

          plt.figure(figsize=(8, 6))  # Adjust figure size as needed
          plt.scatter(slow_outliers[slow_outliers['cluster'] == 0][list(combination)[0]],
                      slow_outliers[slow_outliers['cluster'] == 0][list(combination)[1]],
                      label='Slow Outliers Cluster 0', marker='o', color='red', alpha=0.7)
          plt.scatter(slow_outliers[slow_outliers['cluster'] == 1][list(combination)[0]],
                      slow_outliers[slow_outliers['cluster'] == 1][list(combination)[1]],
                      label='Slow Outliers Cluster 1', marker='x', color='red', alpha=0.7)
          plt.scatter(non_slow_outliers[non_slow_outliers['cluster'] == 0][list(combination)[0]],
                      non_slow_outliers[non_slow_outliers['cluster'] == 0][list(combination)[1]],
                      label='Non-Slow Outliers Cluster 0', marker='o', color='blue', alpha=0.7)
          plt.scatter(non_slow_outliers[non_slow_outliers['cluster'] == 1][list(combination)[0]],
                      non_slow_outliers[non_slow_outliers['cluster'] == 1][list(combination)[1]],
                      label='Non-Slow Outliers Cluster 1', marker='x', color='blue', alpha=0.7)
          cluster_centers = gmm.means_
          plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], marker='*', s=200, c='black', label='Cluster centers')
          plt.title(f'Gaussian Mixture Clusters for {list(combination)[0]} and {list(combination)[1]}')
          plt.xlabel(list(combination)[0])
          plt.ylabel(list(combination)[1])
          plt.legend()
          plt.show()





Asked Gemini to fix the above code and make necessary improvements

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import itertools
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import numpy as np # Might be needed for creating dummy data

# Assuming 'new_df' is the dataframe containing the 5 features you want to use
# And 'slow_outliers', 'non_slow_outliers' are subsets of this dataframe or derived from it.
# These will need to be defined before calling dynamic_clustering.

# Dummy Data Creation (for demonstration purposes - replace with your actual data loading)
# data = {f'feature_{i}': np.random.rand(100) for i in range(5)}
# data['is_outlier'] = np.random.randint(0, 2, 100) # Example outlier flag
# df_full = pd.DataFrame(data)
# slow_outliers = df_full[df_full['is_outlier'] == 1].drop(columns=['is_outlier']).copy()
# non_slow_outliers = df_full[df_full['is_outlier'] == 0].drop(columns=['is_outlier']).copy()
# new_df = df_full.drop(columns=['is_outlier']).copy() # new_df representing the features to choose from

def feature_combinations(df, max_combination_size=2):
    """Generates combinations of features from a DataFrame.

    Args:
        df: The pandas DataFrame containing the features.
        max_combination_size: The maximum size of feature combinations.

    Returns:
        A list of feature combinations (tuples).
    """
    features = df.columns.tolist()
    combinations = []
    # Ensure max_combination_size doesn't exceed the number of features
    for r in range(1, min(max_combination_size + 1, len(features) + 1)):
        combinations.extend(itertools.combinations(features, r))
    return combinations

def plot_clusters(df_outliers, df_non_outliers, combination, clusterer, scaler, cluster_algo_name):
    """Helper function to visualize clusters."""

    feature_names = list(combination)
    n_features = len(combination)

    # Get cluster centers - they are in the scaled space
    if cluster_algo_name == 'kmeans':
        scaled_centers = clusterer.cluster_centers_
    elif cluster_algo_name == 'gmm':
        scaled_centers = clusterer.means_
    else:
        print(f"Warning: Unknown cluster algorithm '{cluster_algo_name}' for plotting.")
        return

    # Inverse transform centers back to the original scale
    cluster_centers = scaler.inverse_transform(scaled_centers)

    plt.figure(figsize=(8, 6))

    if n_features == 1:
        feature = feature_names[0]
        # Plot histograms for the original data
        plt.hist(df_outliers[feature], bins=20, alpha=0.5, label='Outliers', color='red')
        plt.hist(df_non_outliers[feature], bins=20, alpha=0.5, label='Non-Outliers', color='blue')
        plt.title(f'{cluster_algo_name.capitalize()} Distribution of {feature} for Outliers and Non-Outliers')
        plt.xlabel(feature)
        plt.ylabel('Frequency')

        # Plot inverse-transformed cluster centers as vertical lines
        for i, center in enumerate(cluster_centers):
             plt.axvline(center[0], color='black', linestyle='--', label=f'Cluster Center {i}') # Add label per center

    elif n_features == 2:
        feat1, feat2 = feature_names[0], feature_names[1]

        # Scatter plot for outliers, colored by cluster label
        plt.scatter(df_outliers[df_outliers['cluster'] == 0][feat1],
                    df_outliers[df_outliers['cluster'] == 0][feat2],
                    label='Outliers Cluster 0', marker='o', color='red', alpha=0.7)
        plt.scatter(df_outliers[df_outliers['cluster'] == 1][feat1],
                    df_outliers[df_outliers['cluster'] == 1][feat2],
                    label='Outliers Cluster 1', marker='x', color='red', alpha=0.7) # Using 'x' for cluster 1

        # Scatter plot for non-outliers, colored by cluster label
        plt.scatter(df_non_outliers[df_non_outliers['cluster'] == 0][feat1],
                    df_non_outliers[df_non_outliers['cluster'] == 0][feat2],
                    label='Non-Outliers Cluster 0', marker='o', color='blue', alpha=0.7)
        plt.scatter(df_non_outliers[df_non_outliers['cluster'] == 1][feat1],
                    df_non_outliers[df_non_outliers['cluster'] == 1][feat2],
                    label='Non-Outliers Cluster 1', marker='x', color='blue', alpha=0.7) # Using 'x' for cluster 1

        # Plot inverse-transformed cluster centers
        plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], marker='*', s=200, c='black', label='Cluster centers')

        plt.title(f'{cluster_algo_name.capitalize()} Clustering for {feat1} and {feat2}')
        plt.xlabel(feat1)
        plt.ylabel(feat2)

    # Add legend and show plot
    plt.legend()
    plt.grid(True) # Optional: Add grid
    plt.show()

def dynamic_clustering(df_features, df_outliers, df_non_outliers, cluster_type='kmeans', max_combination_size=2):
    """
    Performs dynamic clustering on combinations of features for outlier and non-outlier data,
    and visualizes the results.

    Args:
        df_features (pd.DataFrame): The DataFrame containing the features to be combined.
                                     This should typically include only the relevant features.
        df_outliers (pd.DataFrame): DataFrame containing outlier data. Must contain the features
                                    present in df_features.
        df_non_outliers (pd.DataFrame): DataFrame containing non-outlier data. Must contain the
                                        features present in df_features.
        cluster_type (str): Type of clustering algorithm ('kmeans' or 'gmm'). Defaults to 'kmeans'.
        max_combination_size (int): Maximum size of feature combinations to consider. Defaults to 2.
    """
    if cluster_type.lower() not in ['kmeans', 'gmm']:
        print(f"Error: Invalid cluster_type '{cluster_type}'. Must be 'kmeans' or 'gmm'.")
        return

    # Generate feature combinations from the specified feature dataframe
    combinations = feature_combinations(df_features, max_combination_size)

    if not combinations:
        print("No feature combinations generated. Check input dataframe and max_combination_size.")
        return
    imputer = SimpleImputer(strategy='mean')

    # Keep track of original indices to correctly assign clusters back
    outlier_indices = df_outliers.index
    non_outlier_indices = df_non_outliers.index

    for combination in combinations:
        print(f"\nProcessing combination: {combination} using {cluster_type.upper()}")

        # Select data for the current combination - ensure features exist in both dataframes
        try:
            data_outliers_subset = df_outliers[list(combination)].copy()
            data_non_outliers_subset = df_non_outliers[list(combination)].copy()
        except KeyError as e:
            print(f"Skipping combination {combination}: Feature {e} not found in one of the dataframes.")
            continue

        # Combine data for scaling and clustering
        all_data = pd.concat([data_outliers_subset, data_non_outliers_subset])
        if all_data.isnull().sum().sum() > 0:
            print(f"  -> Handling {all_data.isnull().sum().sum()} missing values in this combination...")
            all_data_imputed = imputer.fit_transform(all_data)
        else:
            print("  -> No missing values found in this combination.")
            all_data_imputed = all_data.values # Convert to numpy array if no imputation happened
        # Convert timedelta columns to total seconds before scaling


        # Preprocess data
        scaler = StandardScaler()
        all_data_scaled = scaler.fit_transform(all_data_imputed) # Fit on combined data for this combination

        # Perform clustering
        if cluster_type.lower() == 'kmeans':
            # Instantiate KMeans inside the loop as it's specific to the data subset
            clusterer = KMeans(n_clusters=2, random_state=42, n_init=10) # Added random_state and n_init
            all_data_clustered = clusterer.fit_predict(all_data_scaled)
            algo_name_for_plot = 'kmeans'
        elif cluster_type.lower() == 'gmm':
            # --- FIX Attempt: Change covariance_type ---
            try:
                # Try with 'full' first (most flexible)
                clusterer = GaussianMixture(n_components=2, random_state=42, covariance_type='full')
                clusterer.fit(all_data_scaled)
                print("  -> GMM fit with 'full' covariance type successful.")
            except ValueError: # Catch potential errors from 'full' like singular covariance
                print("  -> GMM fit with 'full' failed. Trying 'diag' covariance type...")
                try:
                    # If 'full' fails, try 'diag' (diagonal covariance)
                    clusterer = GaussianMixture(n_components=2, random_state=42, covariance_type='diag')
                    clusterer.fit(all_data_scaled)
                    print("  -> GMM fit with 'diag' covariance type successful.")
                except Exception as e: # Catch any other errors
                    print(f"  -> GMM fit with 'diag' also failed: {e}")
                    print(f"  -> Skipping GMM clustering for combination {combination} due to fitting issues.")
                    # Skip plotting for this combination if fit fails
                    continue # Move to the next combination in the loop

            # If we reached here, fit was successful with either 'full' or 'diag'
            all_data_clustered = clusterer.predict(all_data_scaled)
            algo_name_for_plot = 'gmm'

        # Assign cluster labels back to original dataframes using indices
        # This is a temporary assignment for plotting within this loop iteration.
        # Create temporary series to align with original indices
        outlier_clusters = pd.Series(all_data_clustered[:len(data_outliers_subset)], index=data_outliers_subset.index)
        non_outlier_clusters = pd.Series(all_data_clustered[len(data_outliers_subset):], index=data_non_outliers_subset.index)

        # Assign the cluster labels back to the input dataframes for plotting
        # WARNING: This modifies the input dataframes 'df_outliers' and 'df_non_outliers'
        # in place in each iteration. The 'cluster' column will reflect the clustering
        # for the *current* combination being plotted.
        df_outliers['cluster'] = outlier_clusters
        df_non_outliers['cluster'] = non_outlier_clusters


        # Visualize the clusters for this combination
        plot_clusters(df_outliers, df_non_outliers, combination, clusterer, scaler, algo_name_for_plot)

        # Optional: Clean up the temporary 'cluster' column if you don't want it
        # cluttering the original dataframes after the loop finishes.
        # del df_outliers['cluster']
        # del df_non_outliers['cluster']


# Example Usage (assuming you have df_features, slow_outliers, non_slow_outliers DataFrames defined)
# Replace with your actual data loading and splitting logic.
# For example:
# df = pd.read_csv('your_data.csv')
# # Assume 'is_outlier' is a column indicating outliers
# slow_outliers = df[df['is_outlier'] == True].drop(columns=['is_outlier']).copy()
# non_slow_outliers = df[df['is_outlier'] == False].drop(columns=['is_outlier']).copy()
# # Define the 5 features (replace with your actual feature names)
# feature_list = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5']
# df_features = df[feature_list].copy()
# slow_outliers = slow_outliers[feature_list].copy()
# non_slow_outliers = non_slow_outliers[feature_list].copy()


# Then call the function:
# dynamic_clustering(df_features, slow_outliers, non_slow_outliers, cluster_type='kmeans', max_combination_size=2)
# dynamic_clustering(df_features, slow_outliers, non_slow_outliers, cluster_type='gmm', max_combination_size=2)

In [ ]:
# calling the functions to get results for single features
# Kmeans single
dynamic_clustering(new_df, slow_outliers_pca, non_slow_outliers_pca, cluster_type='kmeans', max_combination_size=1)


In [ ]:
# Kmeans pairs
dynamic_clustering(new_df, slow_outliers_pca, non_slow_outliers_pca, cluster_type='kmeans', max_combination_size=2)

In [ ]:
# GMM single
dynamic_clustering(new_df, slow_outliers_pca, non_slow_outliers_pca, cluster_type='gmm', max_combination_size=1)

In [ ]:
# GMM pairs
dynamic_clustering(new_df, slow_outliers_pca, non_slow_outliers_pca, cluster_type='gmm', max_combination_size=2)

## Using the best laptime as a reference for best performance

In [ ]:
weather_df = pd.read_csv('Weather.csv')
weather_df.head()

In [ ]:
weather_df.dtypes

In [ ]:
weather_df.drop(columns=['EventName'], errors='ignore', inplace=True)

In [ ]:
weather_df['Time'] = pd.to_timedelta(weather_df['Time'])
lap_data['Time'] = pd.to_timedelta(lap_data['Time'])

In [ ]:
weather_df.info()

In [ ]:
weather_df = weather_df.drop_duplicates()

In [ ]:
weather_df.head()

In [ ]:
weather_df['RoundNumber'].value_counts()

In [ ]:
Time = weather_df['Time'].values
Time[0]


In [ ]:
TTime = lap_data['Time'].values
TTime[0]

Time scales seem to be different so need to make sure they have same reference point

In [ ]:
offset = lap_data['Time'].min() - weather_df['Time'].min()
weather_df['AdjustedTime'] = weather_df['Time'] + offset
# using merge_asof for time based joining instead of merge to account for the time intervals the data was sampled
lap_data = lap_data.sort_values('Time')
weather_df = weather_df.sort_values('AdjustedTime')
lap_weather_df = pd.merge_asof(lap_data, weather_df, left_on='Time', right_on='AdjustedTime', direction='nearest')
lap_weather_df.head()

In [ ]:
lap_weather_df.columns

In [ ]:
lap_weather_df.head()

In [ ]:
print(lap_weather_df['Time_x'].values[:5])
print(lap_weather_df['Time_y'].values[:5])

Time x seems to be more related to time recorded for lap event. Time y seems to be for matching the weather

In [ ]:
# Dropping irrelevant redundant columns
lap_weather_df.drop(columns=['Time_y', 'RoundNumber_y'], inplace=True)
lap_weather_df.rename(columns={'Time_x': 'Time', 'RoundNumber_x': 'RoundNumber'}, inplace=True)
lap_weather_df.head()

In [ ]:
lap_weather_df.columns

In [ ]:
lap_weather_df.info()

In [ ]:
lap_weather_df = lap_weather_df.sort_values('Time')

plt.figure(figsize=(12, 6))

for driver in lap_weather_df['DriverNumber'].unique():
  driver_data = lap_weather_df[lap_weather_df['DriverNumber'] == driver]
  plt.plot(driver_data['LapNumber'], driver_data['TrackTemp'], marker='o', linestyle='-', label=driver, alpha=0.7)

plt.xlabel('Lap Number')
plt.ylabel('Track Temperature (°C)')
plt.title('Track Temperature across laps for each driver')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()



The plot is a diagnostic to show the noise and patterns after the merge based on nearest timestamp matching. Minor matches may introduce a little noise but data still has non random behavior and no extreme behavior due to extreme outliers

In [ ]:
time_cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
             'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime']

for col in time_cols:
    lap_weather_df[col] = pd.to_timedelta(lap_weather_df[col])

In [ ]:
lap_weather_df = lap_weather_df.drop(columns=['LapStartDate', 'EventName', 'RoundNumber', 'PitOutTime', 'PitInTime', 'LapStartTime', 'AdjustedTime'], errors='ignore')

In [ ]:
lap_weather_df.info()

In [ ]:
# Preparing for Ensemble modeling through EDA and Feature engineering
# EDA
lap_weather_df['LapTimeSeconds'] = lap_weather_df['LapTime'].dt.total_seconds()

Better for models because they don't handle timedelta values too well. Much better to use numerical values like seconds

In [ ]:
lap_weather_df.drop(columns=['LapTime', 'normalized_t'], inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(data=lap_weather_df, x='LapTimeSeconds', hue='DriverNumber', fill=True, common_norm=False, alpha=0.7)
plt.xlabel('Lap Time (seconds)')
plt.ylabel('Density')
plt.title('Distribution of Lap Times by Driver')
plt.show()

In [ ]:
lap_weather_df.info()

In [ ]:
lap_weather_df['LapTimeSeconds'].describe()

In [ ]:
# missing values
missing_vals = lap_weather_df.isnull().mean().sort_values(ascending=False) * 100
missing_vals[missing_vals > 0].to_frame('Missing (%)')

In [ ]:
def drop_low_missing_rows(df, threshold=0.05, exclude_columns=None):
    """
    Drops rows with missing values in columns where the missing rate is below a given threshold.

    Args:
        df (pd.DataFrame): The input dataframe.
        threshold (float): The max missingness allowed to drop rows (e.g., 0.05 = 5%).
        exclude_columns (list): List of column names to exclude from consideration (e.g., ['SpeedI1', 'SpeedST']).

    Returns:
        pd.DataFrame: A cleaned dataframe with selected rows dropped.
    """
    import pandas as pd

    if exclude_columns is None:
        exclude_columns = []

    n_rows_before = df.shape[0]

    # Calculate missingness per column
    missing_percentages = df.isnull().mean()

    # Select columns with missing rate below threshold and not in exclude list
    columns_to_consider = [
        col for col in df.columns
        if missing_percentages[col] <= threshold and col not in exclude_columns
    ]

    # Drop rows where any of those columns are null
    df_cleaned = df.dropna(subset=columns_to_consider)

    n_rows_after = df_cleaned.shape[0]
    print(f"Removed {n_rows_before - n_rows_after} rows ({((n_rows_before - n_rows_after) / n_rows_before):.2%})")
    print(f"Remaining rows: {n_rows_after}")

    return df_cleaned

In [ ]:
low_miss_exclude = ['SpeedI1', 'SpeedST']
drop_low_missing_rows(lap_weather_df, threshold=0.05, exclude_columns=low_miss_exclude)

Removed rows for features that contain less than 5% missing data:

In [ ]:
# group based median imputation for the high percentage missing value features
high_missing_features = ['SpeedST', 'SpeedI1']
for feature in high_missing_features:
  lap_weather_df[feature] = (
      lap_weather_df.groupby('DriverNumber')[feature].transform(lambda x: x.fillna(x.median()))
  )

Very effective imputation technique that is very safe in terms of effects on model results by using medians of subgroups rather than entire data which can cause a loss of context for this type of data

In [ ]:
sector_cols = [
    'Sector1Time', 'Sector2Time', 'Sector3Time',
    'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime'
]

for col in sector_cols:
    lap_weather_df[col + '_Seconds'] = lap_weather_df[col].dt.total_seconds()

lap_weather_df['Time_Seconds'] = lap_weather_df['Time'].dt.total_seconds()

In [ ]:
lap_weather_df.drop(columns=sector_cols, inplace=True)
lap_weather_df.drop(columns=['Time'], inplace=True)

In [ ]:
numerical_features = lap_weather_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = lap_weather_df.select_dtypes(include=['object']).columns

In [ ]:
numerical_features

In [ ]:
categorical_features

In [ ]:
# sector time violin plots
for col in sector_cols[:3]:
  plt.figure(figsize=(10, 6))
  sns.violinplot(data=lap_weather_df, x='DriverNumber', y=col + '_Seconds')
  plt.title(f'Distribution of {col} by Driver')
  plt.xlabel('Driver')

In [ ]:
# sector session time violin plots
for col in sector_cols[3:]:
  plt.figure(figsize=(10, 6))
  sns.violinplot(data=lap_weather_df, x='DriverNumber', y=col + '_Seconds')
  plt.title(f'Distribution of {col} by Driver')
  plt.xlabel('Driver')

In [ ]:
# one plot per speed metric
speed_cols = ['SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST']
df_melted = lap_weather_df[speed_cols].melt(var_name='SpeedType', value_name='SpeedValue')
# Plot
g = sns.FacetGrid(df_melted, col='SpeedType', col_wrap=2, sharex=False, sharey=False, height=4)
g.map(sns.kdeplot, 'SpeedValue', fill=True, common_norm=False, alpha=0.7)
g.set_titles("{col_name}")
plt.tight_layout()
plt.show()

In [ ]:
weather_data = ["TrackTemp", "AirTemp", "Humidity", "WindSpeed", "Pressure"]
plt.figure(figsize=(12, 6))
for col in weather_data:
  plt.hist(lap_weather_df[col], bins=20, alpha=0.5, label=col)
plt.xlabel(f"{col}")
plt.ylabel('Frequency')
plt.title('Distribution of Weather Data')
plt.legend()
plt.show()

In [ ]:
lap_weather_df.info()

In [ ]:
# Tyre/Run dynamics data
Tyre_track_features = ['TyreLife', 'Stint', 'TrackStatus']

In [ ]:
plt.figure(figsize=(12, 6))

# Create the pie chart but without autopct
wedges, texts = plt.pie(
    lap_weather_df['TrackStatus'].value_counts(),
    labels=lap_weather_df['TrackStatus'].unique(),
    # Remove autopct here
    textprops={'fontsize': 10},  # Adjust font size if needed
    startangle=90,  # Optional: Rotate the start angle for better visualization
)

# Create a legend with percentages
plt.legend(
    wedges,
    [f"{l} ({s:1.1f}%)" for l, s in zip(lap_weather_df['TrackStatus'].unique(), lap_weather_df['TrackStatus'].value_counts(normalize=True) * 100)],
    title="Track Status",
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1)  # Adjust position as needed
)

plt.title('Track Status Distribution')
plt.show()

In [ ]:
plt.hist(lap_weather_df['TyreLife'], bins=20, alpha=0.5, label='Tyre Life')
plt.xlabel('Tyre Life')
plt.ylabel('Frequency')
plt.title('Distribution of Tyre Life')
plt.legend()

In [ ]:
stint_counts = lap_weather_df['Stint'].value_counts()
plt.figure(figsize=(12, 6))
plt.bar(stint_counts.index, stint_counts.values)
plt.xlabel('Stint')
plt.ylabel('Count')
plt.title('Stint Distribution')
plt.show()

In [ ]:
# cat feature analysis
plt.figure(figsize=(12, 6))
sns.countplot(data=lap_weather_df, x='Compound', order=lap_weather_df['Compound'].value_counts().index)
plt.title('Distribution of Tire compounds')
plt.xlabel('Tire Compound')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# encoding categorical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
lap_weather_df['Compound'] = le.fit_transform(lap_weather_df['Compound'])

# encoding binary
lap_weather_df['FreshTyre'] = pd.get_dummies(lap_weather_df['FreshTyre'], drop_first=True)
lap_weather_df['Rainfall'] = pd.get_dummies(lap_weather_df['Rainfall'], drop_first=True)
lap_weather_df['IsPersonalBest'] = pd.get_dummies(lap_weather_df['IsPersonalBest'], drop_first=True)
lap_weather_df['IsAccurate'] = pd.get_dummies(lap_weather_df['IsAccurate'], drop_first=True)



In [ ]:
# Make sure to handle potential NaNs in sector times if necessary before summing
# For rows where all sectors are present:
lap_weather_df = lap_weather_df.copy()
lap_weather_df = lap_weather_df.dropna(subset=['Sector1Time_Seconds', 'Sector2Time_Seconds', 'Sector3Time_Seconds', 'LapTimeSeconds'])
lap_weather_df['Calculated_LapTime'] = lap_weather_df['Sector1Time_Seconds'] + lap_weather_df['Sector2Time_Seconds'] + lap_weather_df['Sector3Time_Seconds']

# Compare the calculated sum with the actual LapTimeSeconds
# Check how close they are (accounting for potential float precision issues)
difference = (lap_weather_df['Calculated_LapTime'] - lap_weather_df['LapTimeSeconds']).abs()
print(f"Maximum difference: {difference.max()}")
print(f"Are they essentially equal? {(difference < 1e-6).all()}") # Check if difference is negligible

# Check correlation
correlation_sum = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Calculated_LapTime'])
correlation_s1 = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Sector1Time_Seconds'])
correlation_s2 = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Sector2Time_Seconds'])
correlation_s3 = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Sector3Time_Seconds'])
print(f"Correlation between LapTimeSeconds and Sum of Sectors: {correlation_sum}")
print(f"Correlation between LapTimeSeconds and Sector1Time: {correlation_s1}")
print(f"Correlation between LapTimeSeconds and Sector2Time: {correlation_s2}")
print(f"Correlation between LapTimeSeconds and Sector3Time: {correlation_s3}")

Upon further investigation into lap session vs lap time by looking online they showed they weren't actually performance related measures. So did this correlation analysis to see if this was indeed the case and check whether they cause leakage on the target laptimeseconds

In [ ]:
# Doing the above analysis for sector sesssion laptime
lap_weather_df = lap_weather_df.copy()
lap_weather_df = lap_weather_df.dropna(subset=['Sector1SessionTime_Seconds', 'Sector2SessionTime_Seconds', 'Sector3SessionTime_Seconds', 'LapTimeSeconds'])
lap_weather_df['Calculated_LapTime'] = lap_weather_df['Sector1SessionTime_Seconds'] + lap_weather_df['Sector2SessionTime_Seconds'] + lap_weather_df['Sector3SessionTime_Seconds']
difference = (lap_weather_df['Calculated_LapTime'] - lap_weather_df['LapTimeSeconds']).abs()
print(f"Maximum difference: {difference.max()}")
print(f"Are they essentially equal? {(difference < 1e-6).all()}")
correlation_sum = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Calculated_LapTime'])
correlation_ss1 = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Sector1SessionTime_Seconds'])
correlation_ss2 = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Sector2SessionTime_Seconds'])
correlation_ss3 = lap_weather_df['LapTimeSeconds'].corr(lap_weather_df['Sector3SessionTime_Seconds'])
print(f"Correlation between LapTimeSeconds and Sum of Sectors: {correlation_sum}")
print(f"Correlation between LapTimeSeconds and SectorSession1Time: {correlation_ss1}")
print(f"Correlation between LapTimeSeconds and SectorSession2Time: {correlation_ss2}")
print(f"Correlation between LapTimeSeconds and SectorSession3Time: {correlation_ss3}")

The same analysis for sectorsessionlaptime tells a different story so sectorsessionlaptime is actually important for the task and sectorlaptime is not. But need to do maybe some feature engineering for the time variables

In [ ]:
lap_weather_df.head()

In [ ]:
lap_weather_df.columns

In [ ]:
lap_weather_df = lap_weather_df.drop(columns=['Calculated_LapTime', 'Sector1Time_Seconds', 'Sector2Time_Seconds', 'Sector3Time_Seconds'])

In [ ]:
def safe_data(df):
  """Data to not be tested until Thursday 5/1/2025. This makes sure models can be tested on unseen data"""
  np.random.seed(42)
  test_indices = np.random.choice(df.index, size=int(0.2 * len(df)), replace=False)
  test_data = df.loc[test_indices]
  remaining_data = df.drop(index=test_indices)
  return remaining_data, test_data

In [ ]:
from sklearn.model_selection import KFold
remaining_data, test_data = safe_data(lap_weather_df)
cv_data_x = remaining_data.drop(columns=['LapTimeSeconds', 'DeltaToGlobalBest'])
cv_data_y = remaining_data['DeltaToGlobalBest']

kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf.get_n_splits(cv_data_x)

for i, (train_index, val_index) in enumerate(kf.split(cv_data_x)):
  print(f"Fold {i+1}")
  print(f"Fold {i+1}:")
  print(f"  Train: index={train_index}")
  print(f"  Val: index={val_index}")
  X_train, X_val = cv_data_x.iloc[train_index], cv_data_x.iloc[val_index]
  y_train, y_val = cv_data_y.iloc[train_index], cv_data_y.iloc[val_index]

In [ ]:
driver_best_laptimes = lap_weather_df.groupby('DriverNumber')['LapTimeSeconds'].idxmin()
best_laptime = lap_weather_df.loc[driver_best_laptimes]
lap_weather_df['DeltaToGlobalBest'] = (lap_weather_df['LapTimeSeconds'] - best_laptime['LapTimeSeconds'].min()).abs()

print(f"Global best = {best_laptime['LapTimeSeconds'].min():.3f} s")
print(lap_weather_df['DeltaToGlobalBest'].describe())

In [ ]:
test_data = safe_data()

# Get the indices of the test data
test_indices = test_data.index

# Create the data for cross-validation by excluding the test data
cv_data = lap_weather_df.drop(index=test_indices)

# Now 'cv_data' contains the data to be used for cross-validation,
# and 'test_data' remains your untouched hold-out set.


This is essential for effectively comparing and selecting features relative to this best laptime point. It quantifies the relative performance gap centered around zero and helps with focusing on deviations from the best

In [ ]:
from sklearn.model_selection import KFold
cv_data_x = cv_data.drop(columns=['LapTimeSeconds', 'DeltaToGlobalBest'])
# changing the target to DeltaToGlobalBest
cv_data_y = cv_data['DeltaToGlobalBest']
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf.get_n_splits(cv_data_x)
for i, (train_index, val_index) in enumerate(kf.split(cv_data_x)):
  print(f"Fold {i+1}:")
  print(f"  Train: index={train_index}")
  print(f"  Val: index={val_index}")
  X_train, X_val = cv_data_x.iloc[train_index], cv_data_x.iloc[val_index]
  y_train, y_val = cv_data_y.iloc[train_index], cv_data_y.iloc[val_index]



In [ ]:
# Model 1: Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Identify categorical and numerical features
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()  # specify categorical features
numerical_features = X_train.select_dtypes(exclude=['object']).columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

# Create a pipeline with the preprocessor and the RandomForestRegressor
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = rf_pipeline.predict(X_train)

# Calculate training MSE
train_mse = mean_squared_error(y_train, y_train_pred)
print(f"Random Forest Training MSE: {train_mse}")



# Make predictions on the validation data
y_pred = rf_pipeline.predict(X_val)

# Calculate the mean squared error
val_mse = mean_squared_error(y_val, y_pred)
print(f"Random Forest Validation MSE: {val_mse}")  # 'mse' from previous code


In [ ]:
train_mse_scores = []
val_mse_scores = []
for fold, (train_index, val_index) in enumerate(kf.split(cv_data_x)):
  print(f"Fold {fold + 1}")
  X_train, X_val = cv_data_x.iloc[train_index], cv_data_x.iloc[val_index]
  y_train, y_val = cv_data_y.iloc[train_index], cv_data_y.iloc[val_index]
  rf_pipeline.fit(X_train, y_train)

  y_train_pred = rf_pipeline.predict(X_train)
  train_mse = mean_squared_error(y_train, y_train_pred)
  train_mse_scores.append(train_mse)

  y_pred = rf_pipeline.predict(X_val)
  val_mse = mean_squared_error(y_val, y_pred)
  val_mse_scores.append(val_mse)

  print(f"Random Forest Training MSE: {train_mse}")
  print(f"Random Forest Validation MSE: {val_mse}")

print(f"Average Training MSE: {np.mean(train_mse_scores)}")
print(f"Average Validation MSE: {np.mean(val_mse_scores)}")

In [ ]:
!pip install shap

In [ ]:
# SHAP analysis for random forest
import shap
explainer = shap.TreeExplainer(rf_pipeline.named_steps['regressor'])
shap_values = explainer.shap_values(X_val)
shap_explanation = shap.Explanation(values=shap_values, base_values=explainer.expected_value, data=X_val)
print(f"Expected value (Base Value): {explainer.expected_value}")

The error "ValueError: could not convert string to float: 'Alpine'" is raised because the shap.TreeExplainer is trying to convert the categorical features in X_val to numerical values for calculation, but it encounters a string value ('Alpine' in this case) that it cannot convert. The issue likely stems from categorical features that were not properly encoded or were encoded in the training dataset but not in the validation dataset.

The traceback shows the error originates from line 4 of ipython-input-99-1bab518d47e4 where shap_values = explainer.shap_values(X_val) is called.
shap.TreeExplainer expects numerical input, and X_val likely contains categorical features like 'Team' or 'Driver' that have not been one-hot encoded.
Suggested Changes
# SHAP analysis for random forest
import shap
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer


# Assuming 'X_train' and 'y_train' are already defined

# 1. Identify categorical and numerical features
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_features = X_train.select_dtypes(exclude=['object']).columns.tolist()


# 2. Define pipelines for numerical and categorical features
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)), # Using OrdinalEncoder for SHAP
])

# 3. Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
    ])

# 4. Fit the preprocessor on the training data
preprocessor.fit(X_train)  # Fit on training data to learn encoding

# 5. Transform both training and validation data
X_train_transformed = preprocessor.transform(X_train)
X_val_transformed = preprocessor.transform(X_val)

# 6. Create and train the RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train_transformed, y_train)

# ... (rest of your code for training and evaluation) ...

# 7. SHAP analysis with transformed data
explainer = shap.TreeExplainer(rf_regressor)
shap_values = explainer.shap_values(X_val_transformed)  # Use transformed validation data
shap_explanation = shap.Explanation(values=shap_values, base_values=explainer.expected_value, data=X_val_transformed)
print(f"Expected value (Base Value): {explainer.expected_value}")
Use code with caution
Explanation of Changes:

Import necessary modules: Added imports for ColumnTransformer, Pipeline, StandardScaler, OneHotEncoder, SimpleImputer, and OrdinalEncoder.
Preprocessing Pipeline: Created a preprocessing pipeline using ColumnTransformer to apply different transformations to numerical and categorical features. Imputation is handled for missing values in both types of features. Used OrdinalEncoder instead of OneHotEncoder for categorical encoding. It encodes categories as sequential integers.
Fit and Transform: The preprocessor is fitted on the training data (X_train) to learn the categorical encoding and scaling parameters. Both the training and validation data (X_train, X_val) are transformed using this fitted preprocessor.
SHAP Analysis: The shap.TreeExplainer is created using the trained RandomForestRegressor, and SHAP values are calculated using the transformed validation data (X_val_transformed). This ensures that the data passed to SHAP is in a numerical format, avoiding the ValueError.
Ordinal Encoding for SHAP: Replaced OneHotEncoder with OrdinalEncoder in the categorical pipeline. SHAP often works better with ordinal encoding as it preserves the order of categories, which can be more meaningful for interpretation.
Sources
Rate this answer
I thought in my model training and validation pipeline I took care of that through the categoical_pipeline?

Okay, let's address this point. You are correct that you have a categorical_pipeline in your model training and validation code (ipython-input-95-1bab518d47e4) that includes OneHotEncoder. This pipeline does handle categorical features during training and prediction within the scikit-learn framework.

However, the issue arises when you use shap.TreeExplainer directly on the underlying RandomForestRegressor object (which is accessed via rf_pipeline.named_steps['regressor']).

Here's the key difference:

Scikit-learn Pipeline: When you use the scikit-learn pipeline for training and prediction, the OneHotEncoder within the pipeline is automatically applied to the data before it's passed to the RandomForestRegressor. This ensures the regressor receives numerical data, as required.

SHAP TreeExplainer: The shap.TreeExplainer works by directly accessing the internal structure of the tree-based model (your RandomForestRegressor). It doesn't automatically use the preprocessing steps defined in your scikit-learn pipeline. Therefore, when you pass the original X_val to explainer.shap_values, it encounters the raw categorical data, leading to the ValueError.

In summary: While your training and validation pipeline do handle categorical features correctly for model fitting and prediction, you need to apply the same preprocessing (specifically the categorical encoding) to the data before passing it to shap.TreeExplainer to ensure it receives numerical input.

This is why the suggested changes in my previous response are necessary. They involve applying the preprocessor fitted on the training data to both the training and validation data to ensure consistency and provide the necessary encoding for SHAP analysis. I hope this clarifies the distinction!

In [ ]:
# Shap visualizations
shap.summary_plot(shap_values, X_val, plot_type="bar")
plt.title("SHAP Feature Importance (Mean Absolute SHAP Value)")
plt.show()


In [ ]:
shap.plots.beeswarm(shap_explanation, show=False)
plt.title("SHAP Feature Importance (Bee Swarm Plot)")
plt.show()


# Interpretation:
# - Bar Plot: Longer bars = more important features globally.
# - Beeswarm Plot:
#   - Wider spread = feature has a wide range of impacts.
#   - Color (often default gradient): Red = high feature value, Blue = low feature value.
#   - X-axis position: Positive SHAP value means the feature value pushed the prediction higher (relative to base value), Negative means it pushed it lower.

In [ ]:
shap.dependence_plot("Compound", shap_values, X_val, interaction_index="auto", show=False)
plt.title("SHAP Dependence Plot for Compound")
plt.show()


# Interpretation:
# - X-axis: Value of the feature being plotted.
# - Y-axis: SHAP value for that feature (its impact on the prediction).
# - Vertical scatter/patterns: Suggest interactions with other features. Color helps identify the interacting feature.

In [ ]:
i = 0  # index in X_explain
shap.force_plot(
    explainer.expected_value,
    shap_values[i],
    X_explain.iloc[i],
    matplotlib=True
)
plt.show()

# Interpretation:
# - Base value: The average prediction (explainer.expected_value).
# - Red arrows: Features pushing the prediction higher than the base value.
# - Blue arrows: Features pushing the prediction lower than the base value.
# - Arrow width: Magnitude of the feature's impact for this specific prediction.
# - Final prediction: Base value + sum of all feature contributions (SHAP values).

In [ ]:
mean_abs_shap = abs(shap_values).mean(axis=0)
if X_val.columns:
  feature_importance_df = pd.DataFrame({
      'feature' : X_val.columns,
      'mean_abs_shap' : mean_abs_shap
  })
  feature_importantce_df = feature_importance_df.sort_values(by='mean_abs_shap', ascending=False)
  print("\nFeature Importance based on Mean Absolute SHAP values:")
  print(feature_importance_df)
else:
  print("\nMean Absolute SHAP values (feature order corresponds to columns in X_val):")
  print(sorted(mean_abs_shap, reverse=True))

In [ ]:
top_n_features = 10
if X_val.columns:
  top_features = feature_importance_df.head(top_n_features)['feature'].head(top_n_features).tolist()
  print(f"\nTop {top_n_features} Most Important Features:")
  print(top_features)

In [ ]:
# Model 2: XGBoost
!pip install xgboost
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"XGBoost MSE: {mse}")

In [ ]:
# SHAP analysis for XGboost
exp = shap.TreeExplainer(xgb_model)
shap_values_xgb = exp.shap_values(X_val)
shap_explanation = shap.Explanation(values=shap_values, base_values=exp.expected_value, data=X_val)
print(f"Expected value (Base Value): {exp.expected_value}")

In [ ]:
# shap visualizations
shap.summary_plot(shap_values_xgb, X_val, plot_type="bar")
plt.title("SHAP Feature Importance (Mean Absolute SHAP Value) XGB")
plt.show()

In [ ]:
shap.plots.beeswarm(shap_explanation, show=False)
plt.title("SHAP Feature Importance (Bee Swarm Plot) XGB")
plt.show()

In [ ]:
shap.dependence_plot("Compound", shap_values_xgb, X_val, interaction_index="auto", show=False)
plt.title("SHAP Dependence Plot for Compound XGB")
plt.show()

In [ ]:
i = 0
shap.force_plot(exp.expected_value, shap_values[i], X_explain.iloc[i], matplotlib=True)
plt.show()

In [ ]:
# SHAP feature selection
mean_abs_shap = abs(shap_values_xgb).mean(axis=0)
if X_val.columns:
  feature_importance_df_xgb = pd.DataFrame({
      'feature' : X_val.columns,
      'mean_abs_shap' : mean_abs_shap
  })
  feature_importantce_df_xgb = feature_importance_df_xgb.sort_values(by='mean_abs_shap', ascending=False)
  print("\nFeature Importance based on Mean Absolute SHAP values:")
  print(feature_importance_df_xgb)
else:
  print("\nMean Absolute SHAP values (feature order corresponds to columns in X_val):")
  print(sorted(mean_abs_shap, reverse=True))


In [ ]:
top_n_features = 10
if X_val.columns:
  top_features = feature_importance_df_xgb.head(top_n_features)['feature'].head(top_n_features).tolist()
  print(f"\nTop {top_n_features} Most Important Features:")
  print(top_features)

In [ ]:
!pip install catboost

In [ ]:
# Model 3: Catboost
from catboost import CatBoostRegressor
cbr_model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, loss_function='RMSE', random_state=42)
cbr_model.fit(X_train, y_train)
y_pred = cbr_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"CatBoost MSE: {mse}")

In [ ]:
# SHAP analysis for catboost
exp = shap.TreeExplainer(cbr_model)
shap_values_cbr = exp.shap_values
shap_explanation = shap.Explanation(values=shap_values, base_values=exp.expected_value, data=X_val)
print(f"Expected value (Base Value): {exp.expected_value}")

In [ ]:
shap.summary_plot(shap_values_cbr, X_val, plot_type="bar")
plt.title("SHAP Feature Importance (Mean Absolute SHAP Value) CatBoost")
plt.show()

In [ ]:
shap.plots.beeswarm(shap_explanation, show=False)
plt.title("SHAP Feature Importance (Bee Swarm Plot) CatBoost")
plt.show()

In [ ]:
shap.dependence_plot("Compound", shap_values_cbr, X_val, interaction_index="auto", show=False)
plt.title("SHAP Dependence Plot for Compound CatBoost")
plt.show()

In [ ]:
i = 0
shap.force_plot(exp.expected_value, shap_values[i], X_explain.iloc[i], matplotlib=True)
plt.show()

In [ ]:
# shap feature selection
mean_abs_shap = abs(shap_values_cbr).mean(axis=0)
if X_val.columns:
  feature_importance_df_cbr = pd.DataFrame({
      'feature' : X_val.columns,
      'mean_abs_shap' : mean_abs_shap
  })
  feature_importantce_df_cbr = feature_importance_df_cbr.sort_values(by='mean_abs_shap', ascending=False)
  print("\nFeature Importance based on Mean Absolute SHAP values:")
  print(feature_importance_df_cbr)
else:
  print("\nMean Absolute SHAP values (feature order corresponds to columns in X_val):")
  print(sorted(mean_abs_shap, reverse=True))

In [ ]:
top_n_features = 10
if X_val.columns:
  top_features = feature_importance_df_cbr.head(top_n_features)['feature'].head(top_n_features).tolist()
  print(f"\nTop {top_n_features} Most Important Features:")
  print(top_features)